In [62]:
# qibo's
import qibo
from qibo import models

# boostvqe's
from boostvqe import ansatze

# pytket
import pytket.qasm
from pytket.circuit.display import render_circuit_jupyter
from sympy import Symbol
from pytket import Circuit

import numpy as np
from copy import deepcopy

In [63]:
qibo.set_backend("tensorflow")

[Qibo 0.2.12|INFO|2024-10-17 13:19:56]: Using tensorflow backend on /device:CPU:0


## Qibo "symbolic" circuit structure

In [64]:
# build ansatz circuit
nlayer = 1
nqubits = 5
ansatz_circ = ansatze.hdw_efficient(nqubits, nlayer)
print(ansatz_circ.draw())
print(type(ansatz_circ))

[Qibo 0.2.12|WARNING|2024-10-17 13:19:56]: Starting on qibo 0.2.13, ``Circuit.draw`` will work in-place. The in-place method is currently implemented as ``Circuit.display``, but will be renamed as ``Circuit.draw`` on release 0.2.13. In release 0.2.12, the in-place display of circuits is accessible as ``Circuit.display``.


q0: ─RY─RZ─o───RY─RZ───o─RY─
q1: ─RY─RZ─Z───RY─RZ─o─|─RY─
q2: ─RY─RZ───o─RY─RZ─Z─|─RY─
q3: ─RY─RZ───Z─RY─RZ───|─RY─
q4: ─RY─RZ─────RY─RZ───Z─RY─
<class 'qibo.models.circuit.Circuit'>


From the definition of the ansatze in `boostvqe`, we can see that all the $\theta$ are set to $0$. When we convert the circuit - QASM - Tket, we will see that the circuit has all the params $\theta=0$ as expected.

In [65]:
ansatz_circ_qasm = models.Circuit.to_qasm(ansatz_circ)
ansatz_circ_tket = pytket.qasm.circuit_from_qasm_str(ansatz_circ_qasm)
render_circuit_jupyter(ansatz_circ_tket)

Next, we show how Qibo updates the parameters in a "symbolic ciruict". For this, we need to go to definition of `train_vqe` and from there `minimize` in `utils.py`.

In [66]:
# initial params
params_len = len(ansatz_circ.get_parameters())
# fix numpy seed to ensure replicability of the experiment
seed = 10
np.random.seed(seed)
initial_params = np.random.uniform(-np.pi, np.pi, params_len)
print(initial_params)
qc = deepcopy(ansatz_circ)
qc.set_parameters(initial_params)
qc_qasm = models.Circuit.to_qasm(qc)
qc_tket = pytket.qasm.circuit_from_qasm_str(qc_qasm)
render_circuit_jupyter(qc_tket)

[ 1.70475788 -3.01120431  0.83973663  1.5632809  -0.00938072 -1.72915367
 -1.89712697  1.63696274 -2.07903793 -2.58653723  1.16465009  2.84875441
 -3.11678496  0.07660625  1.96425543  0.70702213  1.39332975 -1.30768123
  2.62495223  1.34821941  0.26731415 -2.2483119  -0.79582348  1.09411377
 -0.36547294]


We see that Qibo uses ordinary circuits with a method `set_parameters` to reset all the variables columns by columns (depth), ignoring the gates without free parameters (CZ).

## Pytket symbolic circuit structure

This section references the Nexus VQE tutorial (link: https://docs.quantinuum.com/nexus/trainings/notebooks/vqe_example.html)


In [68]:
symbols = [Symbol(f"p{i}") for i in range(4)]
symbolic_circuit = Circuit(nqubits)
symbolic_circuit.X(0)
symbolic_circuit.Ry(symbols[0], 0).Ry(symbols[1], 1)
symbolic_circuit.CX(0, 1)
symbolic_circuit.Ry(symbols[2], 0).Ry(symbols[3], 0)
render_circuit_jupyter(symbolic_circuit)

We see here that the symbolic circuit is defined within the same `Circuit` class, but the parameters are set via the `Symbol` in `sympy` package.

Therefore, we wonder if we can use `set_parameters` in Qibo to convert ordinary circuit to symbolic circuit.

In [93]:
symbols = [Symbol(f"p{i}") for i in range(params_len)]
qc = deepcopy(ansatz_circ)
qc.set_parameters(symbols)
qc_qasm = models.Circuit.to_qasm(qc)
qc_tket = pytket.qasm.circuit_from_qasm_str(qc_qasm)
render_circuit_jupyter(qc_tket)
print(len(qc_tket.free_symbols()))

25


This method indeed converts our ansatze circuit into a symbolic one and the length of free parameters is correct. However, the symbols are attached with an extra "/pi". To see whether this extra factor affects our values, we can set all parameters to $1$.

In [94]:
params = [1 for i in range(params_len)]
symbol_dict = {s: p for s, p in zip(qc_tket.free_symbols(), params)}
qc_tket.symbol_substitution(symbol_dict)
render_circuit_jupyter(qc_tket)

In [95]:
qc_tket.get_commands()[0].op

Ry(0.31831)

From this, we can conclude that:
1. Qibo circuits can easily be converted into a symbolic one and hence be cast into quantinuum symbolic circuit.
2. However, after casting into Qasm and then Pytket, the parameters have an extra factor $/\pi$, e.g., to have $Rz(2\pi)$ in the converted circuit, the parameter needs to be $2$ instead of $2\pi$.

Let's check what could be causing this:
Definition of Rz gates in Qibo

$$
Rz(\theta) = 
\begin{pmatrix}
e^{-i\theta/2}& 0\\ 0 & e^{i\theta/2}
\end{pmatrix}
$$

whereas in pytket:

$$
Rz(\theta) = 
\begin{pmatrix}
e^{-i\pi\theta/2}& 0\\ 0 & e^{i\pi\theta/2}
\end{pmatrix}.
$$

This means that the casting functions we have are correctly converting between Qibo circuits and Pytket circuits considering the difference in definition. Nevertheless, the user need to be cautious when handling the parameters between the two platforms.